# 1 - Teste inicial de Modelo

Introduction
In this activity, you will apply the concepts learned in the previous lessons by implementing an ML model from scratch. This hands-on exercise will guide you through the process of building, training, and evaluating a model using your choice of ML frameworks—TensorFlow, PyTorch, or Scikit-learn. The goal is to reinforce your understanding of model implementation. By the end of this activity, you will gain practical experience in deploying a working model.

By the end of this activity, you will:

Build an ML model using a specified framework.

Train the model on a provided dataset.

Evaluate the model’s performance.

Save and potentially reload the model for future use.

Dataset
You will use the CIFAR-10 dataset, a well-known dataset consisting of 60,000 32x32 color images in 10 different classes. The dataset is already split into 50,000 training images and 10,000 test images. You can download the dataset directly through the TensorFlow or PyTorch libraries, or if using Scikit-learn, you may 
download it separately
.

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:11<00:00, 14.8MB/s] 


In [5]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()



In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[Epoch 1, Batch 2000] loss: 2.205
[Epoch 1, Batch 4000] loss: 1.852
[Epoch 1, Batch 6000] loss: 1.643
[Epoch 1, Batch 8000] loss: 1.561
[Epoch 1, Batch 10000] loss: 1.499
[Epoch 1, Batch 12000] loss: 1.436
[Epoch 2, Batch 2000] loss: 1.403
[Epoch 2, Batch 4000] loss: 1.350
[Epoch 2, Batch 6000] loss: 1.353
[Epoch 2, Batch 8000] loss: 1.340
[Epoch 2, Batch 10000] loss: 1.313
[Epoch 2, Batch 12000] loss: 1.293
[Epoch 3, Batch 2000] loss: 1.237
[Epoch 3, Batch 4000] loss: 1.241
[Epoch 3, Batch 6000] loss: 1.211
[Epoch 3, Batch 8000] loss: 1.225
[Epoch 3, Batch 10000] loss: 1.204
[Epoch 3, Batch 12000] loss: 1.214
[Epoch 4, Batch 2000] loss: 1.114
[Epoch 4, Batch 4000] loss: 1.143
[Epoch 4, Batch 6000] loss: 1.143
[Epoch 4, Batch 8000] loss: 1.147
[Epoch 4, Batch 10000] loss: 1.126
[Epoch 4, Batch 12000] loss: 1.144
[Epoch 5, Batch 2000] loss: 1.049
[Epoch 5, Batch 4000] loss: 1.057
[Epoch 5, Batch 6000] loss: 1.067
[Epoch 5, Batch 8000] loss: 1.077
[Epoch 5, Batch 10000] loss: 1.079
[Epoc

In [13]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [14]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

Accuracy of the network on the 10000 test images: 61.62 %
